In [ ]:
content = event["queryStringParameters"]["content"]

res_body = {}
res_body["content"] = content

http_res = {}
http_res["statusCode"] = 200
http_res["headers"] = {}
http_res["headers"]["Content-Type"] = "application/json"
http_res["body"] = json.dumps(res_body)
return http_res

Heatmap for correlations of features

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


def plot_correlation_heatmap(df: pd.core.frame.DataFrame, title_name: str='Train correlation') -> None:

    corr = df.corr()
    fig, axes = plt.subplots(figsize=(12, 8))
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(corr, mask=mask, linewidths=.5, cmap='RdBu_r', annot=True,annot_kws={"size": 8})
    plt.title(title_name)
    plt.show()


plot_correlation_heatmap(df, 'Train Dataset Correlation')

Better Summary

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['average'] = desc['mean'].values
    summ['standard_deviation'] = desc['std'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

summary(train)

Base class

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
class DataPrep:
    def __init__(self, train, test, idcol, target_col):
        self.train_ids = train #just the id column
        self.train_data = train #everything from train
        self.train = train.drop(columns=[idcol, target_col]) #just the data
        self.train_target = train[target_col] #Just the target
        self.test_ids = test.id #Test IDs
        self.test = test.drop(columns=[idcol]) #Just the test data
        self.normalize()
        self.SKfold(10)

    def SKfold(self, splits):
        self.skf = StratifiedKFold(n_splits=splits, shuffle=True)
        
    def drop_cols(self, columns):
        self.train = self.train.drop(columns=columns)
        self.test = self.test.drop(columns=columns)

    def normalize(self):
        self.ss = StandardScaler()
        new_train = self.ss.fit_transform(self.train)
        self.scaled_train = pd.DataFrame(new_train, index=self.train.index, columns=self.train.columns)
        new_test = self.ss.transform(self.test)
        self.scaled_test = pd.DataFrame(new_test, index=self.test.index, columns=self.test.columns)

## Use cross_val_score and cross_val predict from sklearn.model_selection to skip the having to do the kfolds.

Using Optuna for Optimization?